### КУРС "Введение в обработку естественного языка"

ДЗ к Уроку 11. Модель Transformer-1

Разобраться с моделькой перевода (с механизмом внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

In [1]:
!pip install transformers

In [2]:
!pip install transformers[sentencepiece]

In [3]:
import time
import tensorflow as tf

#### Transformers 
Попробуем трансформеры в машинном переводе.
Возьмем самый простой пример, с готовой обученной моделью. 

In [4]:
%%time
from transformers import pipeline
translator = pipeline('translation_en_to_de')
text = 'Hello world! This is robot, and it would be show that do it correct!'
translation = translator(text)
print(translation)

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

C:\Users\ASUS\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

C:\Users\ASUS\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[{'translation_text': 'Hallo Welt, das ist Roboter, und es wäre zu zeigen, dass es richtig tun!'}]
Wall time: 6min 27s


Однако для русского языка простой pipeline не сработает.
Для этой задачи надо сделать кастомный pipeline.

In [5]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
model = TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

C:\Users\ASUS\anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/307M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-ru-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [6]:
%%time
text = 'Привет! Это я и я собираюсь показать вам как делать это правильно!'
tokenized_text = tokenizer.prepare_seq2seq_batch([text])
translation = model.generate(**tokenized_text)
translation_text = tokenizer.batch_decode(translation, skip_special_tokens=True)[0]
print(translation_text)

C:\Users\ASUS\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
C:\Users\ASUS\anaconda3\lib\site-packages\transformers\generation\tf_utils.py:1800: UserWarning: Neither `max_length` nor `max

It's me and I going to show you how to do it right!
Wall time: 2min 23s


In [7]:
dostoevsky = """
                В конце ноября, в оттепель, часов в девять утра, поезд Петербургско-Варшавской железной дороги на всех парах подходил к Петербургу. Было так сыро и туманно, что насилу рассвело; в десяти шагах, вправо и влево от дороги, трудно было разглядеть хоть что-нибудь из окон вагона. Из пассажиров были и возвращавшиеся из-за границы; но более были наполнены отделения для третьего класса, и все людом мелким и деловым, не из очень далека. Все, как водится, устали, у всех отяжелели за ночь глаза, все назяблись, все лица были бледножелтые, под цвет тумана.

В одном из вагонов третьего класса, с рассвета, очутились друг против друга, у самого окна, два пассажира, - оба люди молодые, оба почти налегке, оба не щегольски одетые, оба с довольно замечательными физиономиями, и оба пожелавшие, наконец, войти друг с другом в разговор. Если б они оба знали один про другого, чем они особенно в эту минуту замечательны, то, конечно, подивились бы, что случай так странно посадил их друг против друга в третьеклассном вагоне петербургско-варшавского поезда. Один из них был небольшого роста, лет двадцати семи, курчавый и почти черноволосый, с серыми, маленькими, но огненными глазами. Нос его был широки сплюснут, лицо скулистое; тонкие губы беспрерывно складывались в какую-то наглую, насмешливую и даже злую улыбку; но лоб его был высок и хорошо сформирован и скрашивал неблагородно развитую нижнюю часть лица. Особенно приметна была в этом лице его мертвая бледность, придававшая всей физиономии молодого человека изможденный вид, несмотря на довольно крепкое сложение, и вместе с тем что-то страстное, до страдания, не гармонировавшее с нахальною и грубою улыбкой и с резким, самодовольным его взглядом. Он был тепло одет, в широкий, мерлушечий, черный, крытый тулуп, и за ночь не зяб, тогда как сосед его принужден был вынести на своей издрогшей спине всю сладость сырой, ноябрьской русской ночи, к которой, очевидно, был не приготовлен. На нем был довольно широкий и толстый плащ без рукавов и с огромным капюшоном, точь-в-точь как употребляют часто дорожные, по зимам, где-нибудь далеко за границей, в Швейцарии, или, например, в Северной Италии, не рассчитывая, конечно, при этом и на такие концы по дороге, как от Эйдкунена до Петербурга. Но что годилось и вполне удовлетворяло в Италии, то оказалось не совсем пригодным в России. Обладатель плаща с капюшоном был молодой человек, тоже лет двадцати шести или двадцати семи, роста немного повыше среднего, очень белокур, густоволос, со впалыми щеками и с легонькою, востренькою, почти совершенно белою бородкой. Глаза его были большие, голубые и пристальные; во взгляде их было что-то тихое, но тяжелое, что-то полное того странного выражения, по которому некоторые угадывают с первого взгляда в субъекте падучую болезнь. Лицо молодого человека было, впрочем, приятное, тонкое и сухое, но бесцветное, а теперь даже до-синя иззябшее. В руках его болтался тощий узелок из старого, полинялого фуляра, заключавший, кажется, все его дорожное достояние. На ногах его были толстоподошвенные башмаки с штиблетами, - все не по-русски. Черноволосый сосед в крытом тулупе все это разглядел, частию от нечего делать, и, наконец, спросил с тою неделикатною усмешкой, в которой так бесцеремонно и небрежно выражается иногда людское удовольствие при неудачах ближнего:

- Зябко?

И повел плечами.

- Очень, - ответил сосед с чрезвычайною готовностью, - и заметьте, это еще оттепель. Что ж, если бы мороз? Я даже не думал, что у нас так холодно. Отвык.

- Из-за границы что ль?

- Да, из Швейцарии.

- Фью! Эк ведь вас!..

Черноволосый присвистнул и захохотал.

Завязался разговор. Готовность белокурого молодого человека в швейцарском плаще отвечать на все вопросы своего черномазого соседа была удивительная и без всякого подозрения совершенной небрежности, неуместности и праздности иных вопросов. Отвечая, он объявил, между прочим, что действительно долго не был в России, слишком четыре года, что отправлен был за границу по болезни, по какой-то странной нервной болезни, в роде падучей или Виттовой пляски, каких-то дрожаний и судорог. Слушая его, черномазый несколько раз усмехался; особенно засмеялся он, когда на вопрос: "что же, вылечили?" - белокурый отвечал, что "нет, не вылечили".

- Хе! Денег что, должно быть, даром переплатили, а мы-то им здесь верим, - язвительно заметил черномазый.

- Истинная правда! - ввязался в разговор один сидевший рядом и дурно одетый господин, нечто в роде закорузлого в подьячестве чиновника, лет сорока, сильного сложения, с красным носом и угреватым лицом: - истинная правда-с, только все русские силы даром к себе переводят!

- О, как вы в моем случае ошибаетесь, - подхватил швейцарский пациент, тихим и примиряющим голосом; - конечно, я спорить не могу, потому что всего не знаю, но мой доктор мне из своих последних еще на дорогу сюда дал, да два почти года там на свой счет содержал.

- Что ж, некому платить что ли было? - спросил черномазый.

- Да, господин Павлищев, который меня там содержал, два года назад помер; я писал потом сюда генеральше Епанчиной, моей дальней родственнице, но ответа не получил. Так с тем и приехал.

- Куда же приехали-то?

- То-есть, где остановлюсь?.. Да не знаю еще, право… так…

- Не решились еще?

И оба слушателя снова захохотали.

- И небось в этом узелке вся ваша суть заключается? - спросил черномазый.
"""

In [8]:
%%time
text = dostoevsky
tokenized_text = tokenizer.prepare_seq2seq_batch([text])
translation = model.generate(**tokenized_text)
translation_text = tokenizer.batch_decode(translation, skip_special_tokens=True)[0]
print(translation_text)

At the end of November, in the throes, at nine o'clock in the morning, the Petersburg-Warshav railway train on all the pairs approached Petersburg. It was so damp and foggy that his eyes were so thin; ten feet to the right and to the left of the road, it was hard to see anything out of the windows of the wagon grudgingly. Out of the passengers came back from abroad; but they were filled with branches for the third class, and all men were very small and business, not very far apart. Everything was out of the blue, tired, all over the night, all eyes were heavy, all the faces were pale, all the faces were pale, were in the colour of the fog. In one of the third class wagons, out of the sun, came out against each other, out of the height of the window, two passengers, both of them were young, both of them were easy, both of them were not lightly dressed, both of them were rather stunned in the night, all of them, all of them were pale, all of them were pale, finally in the colours, in one